In [2]:
import prjmod as pm
import prjmod.commons as commons
from pyspark.sql import SparkSession
import pyspark
import pandas as pd

In [3]:

spark = SparkSession.builder.getOrCreate()

In [4]:
# Print the tables in the catalog
print(spark.catalog.listTables())

[]


# Reading Data 

## Local file to Spark DataFrame (SDF)

In [75]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, FloatType

csv_schema = StructType([ \
    StructField("_c0",StringType(),True), \
    StructField("year",IntegerType(),True), \
    StructField("month",IntegerType(),True), \
    StructField("day", IntegerType(), True), \
    StructField("dep_time", FloatType(), True), \
    StructField("sched_dep_time", IntegerType(), True), \
    StructField("dep_delay",FloatType(),True), \
    StructField("arr_time",FloatType(),True), \
    StructField("sched_arr_time",IntegerType(),True), \
    StructField("arr_delay", FloatType(), True), \
    StructField("carrier", StringType(), True), \
    StructField("flight", StringType(), True), \
    StructField("tailnum", StringType(), True), \
    StructField("origin", StringType(), True), \
    StructField("dest",StringType(),True), \
    StructField("air_time",FloatType(),True), \
    StructField("distance",IntegerType(),True), \
    StructField("hour", IntegerType(), True), \
    StructField("minute", IntegerType(), True), \
    StructField("time_hour", TimestampType(), True) \
  ])



In [76]:

sdf_flights.columns

['_c0',
 'year',
 'month',
 'day',
 'dep_time',
 'sched_dep_time',
 'dep_delay',
 'arr_time',
 'sched_arr_time',
 'arr_delay',
 'carrier',
 'flight',
 'tailnum',
 'origin',
 'dest',
 'air_time',
 'distance',
 'hour',
 'minute',
 'time_hour']

In [77]:
sdf_flights = spark.read.csv(commons.DL_FILE_FLIGHTS, header=True, sep = ';', schema = csv_schema)
# sdf_flights = spark.read.csv(commons.DL_FILE_FLIGHTS, header=True, sep = ';')

In [78]:
sdf_flights.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: float (nullable = true)
 |-- sched_dep_time: integer (nullable = true)
 |-- dep_delay: float (nullable = true)
 |-- arr_time: float (nullable = true)
 |-- sched_arr_time: integer (nullable = true)
 |-- arr_delay: float (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: float (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- time_hour: timestamp (nullable = true)



In [79]:
sdf_flights.show()

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|_c0|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|  1|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|
|  2|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|
|  3|2013|    1|  1|   542.0|           540|      2.0|   923.0|           850|     33.0|     AA|  1141| N619AA|   JFK| MIA|   160.

In [80]:
print(sdf_flights.describe())

DataFrame[summary: string, _c0: string, year: string, month: string, day: string, dep_time: string, sched_dep_time: string, dep_delay: string, arr_time: string, sched_arr_time: string, arr_delay: string, carrier: string, flight: string, tailnum: string, origin: string, dest: string, air_time: string, distance: string, hour: string, minute: string]


In [81]:
# Add sdf_flights to the catalog
sdf_flights.createOrReplaceTempView("sdf_flights_temp")

print(spark.catalog.listTables())

[Table(name='sdf_flights_temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [82]:
# Now we can get the data as SDF from the table in the Spark cluster:
sdf_flights_frm_table = spark.table('sdf_flights_temp')

## SDF to pandas DataFrame (PDF)

In [83]:
pdf_flights = sdf_flights.toPandas()

In [84]:
print(pdf_flights.head())

  _c0  year  month  day  dep_time  sched_dep_time  dep_delay  arr_time  \
0   1  2013      1    1     517.0             515        2.0     830.0   
1   2  2013      1    1     533.0             529        4.0     850.0   
2   3  2013      1    1     542.0             540        2.0     923.0   
3   4  2013      1    1     544.0             545       -1.0    1004.0   
4   5  2013      1    1     554.0             600       -6.0     812.0   

   sched_arr_time  arr_delay carrier flight tailnum origin dest  air_time  \
0             819       11.0      UA   1545  N14228    EWR  IAH     227.0   
1             830       20.0      UA   1714  N24211    LGA  IAH     227.0   
2             850       33.0      AA   1141  N619AA    JFK  MIA     160.0   
3            1022      -18.0      B6    725  N804JB    JFK  BQN     183.0   
4             837      -25.0      DL    461  N668DN    LGA  ATL     116.0   

   distance  hour  minute           time_hour  
0      1400     5      15 2013-01-01 05:00:0

In [85]:
pdf_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   _c0             336776 non-null  object        
 1   year            336776 non-null  int32         
 2   month           336776 non-null  int32         
 3   day             336776 non-null  int32         
 4   dep_time        328521 non-null  float32       
 5   sched_dep_time  336776 non-null  int32         
 6   dep_delay       328521 non-null  float32       
 7   arr_time        328063 non-null  float32       
 8   sched_arr_time  336776 non-null  int32         
 9   arr_delay       327346 non-null  float32       
 10  carrier         336776 non-null  object        
 11  flight          336776 non-null  object        
 12  tailnum         336776 non-null  object        
 13  origin          336776 non-null  object        
 14  dest            336776 non-null  obj

## Pandas to SDF

In [86]:
# Create sdf_flights_2 from pd_temp
sdf_flights_2 = spark.createDataFrame(pdf_flights.iloc[0:100, :], schema=csv_schema)


In [87]:
sdf_flights_2.show()

Py4JJavaError: An error occurred while calling o259.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 46.0 failed 1 times, most recent failure: Lost task 0.0 in stage 46.0 (TID 85) (ESW5CG8461BGN.mshome.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOr

In [88]:
# Examine the tables in the catalog
print(spark.catalog.listTables())

[Table(name='sdf_flights_temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [89]:

# Add sdf_flights_2 to the catalog
sdf_flights_2.createOrReplaceTempView("sdf_flights_2_temp")

# Examine the tables in the catalog again
print(spark.catalog.listTables())

[Table(name='sdf_flights_2_temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='sdf_flights_temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


# Querying Spark Tables with SQL Spark

In [90]:
flights10 = spark.sql("SELECT * FROM sdf_flights_temp LIMIT 10")

DataFrame[_c0: string, year: int, month: int, day: int, dep_time: float, sched_dep_time: int, dep_delay: float, arr_time: float, sched_arr_time: int, arr_delay: float, carrier: string, flight: string, tailnum: string, origin: string, dest: string, air_time: float, distance: int, hour: int, minute: int, time_hour: timestamp]


In [91]:
flights10.show()

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|_c0|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|  1|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|
|  2|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|
|  3|2013|    1|  1|   542.0|           540|      2.0|   923.0|           850|     33.0|     AA|  1141| N619AA|   JFK| MIA|   160.

## Filtering

In [94]:
# Filter flights by passing a string
long_flights1 = sdf_flights.filter("distance > 1000")

# Filter flights by passing a column of boolean values
long_flights2 = sdf_flights.filter(sdf_flights.distance > 1000)

# Print the data to check they're equal
long_flights1.show(10)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|_c0|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|  1|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|
|  2|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|
|  3|2013|    1|  1|   542.0|           540|      2.0|   923.0|           850|     33.0|     AA|  1141| N619AA|   JFK| MIA|   160.

In [96]:
long_flights2.show(10)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|_c0|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|  1|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|
|  2|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|
|  3|2013|    1|  1|   542.0|           540|      2.0|   923.0|           850|     33.0|     AA|  1141| N619AA|   JFK| MIA|   160.

## Selecting

In [98]:
# Select the first set of columns
selected1 = sdf_flights.select("tailnum", "origin", "dest")

# Select the second set of columns
temp = sdf_flights.select(sdf_flights.origin, sdf_flights.dest, sdf_flights.carrier)

# Define first filter
filterA = sdf_flights.origin == "SEA"

# Define second filter
filterB = sdf_flights.dest == "PDX"

# Filter the data, first by filterA then by filterB
selected2 = temp.filter(filterA).filter(filterB)

In [100]:
# Define avg_speed
avg_speed = (sdf_flights.distance/(sdf_flights.air_time/60)).alias("avg_speed")

# Select the correct columns
speed1 = sdf_flights.select("origin", "dest", "tailnum", avg_speed)

# Create the same table using a SQL expression
speed2 = sdf_flights.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed")

## Aggregating



In [107]:
# Find the shortest flight from PDX in terms of distance
sdf_flights.filter(sdf_flights.origin == 'JFK').groupBy().min('distance').show()

# Find the longest flight from SEA in terms of air time
sdf_flights.filter(sdf_flights.origin == 'JFK').groupBy().max('air_time').show()

+-------------+
|min(distance)|
+-------------+
|           94|
+-------------+

+-------------+
|max(air_time)|
+-------------+
|        691.0|
+-------------+



In [109]:
# Average duration of Delta flights
sdf_flights.filter(sdf_flights.carrier == "DL").filter(sdf_flights.origin == "JFK").groupBy().avg("air_time").show()

# Total hours in the air
sdf_flights.withColumn("duration_hrs", sdf_flights.air_time/60).groupBy().sum("duration_hrs").show()

+------------------+
|     avg(air_time)|
+------------------+
|229.81132350795272|
+------------------+

+-----------------+
|sum(duration_hrs)|
+-----------------+
|822110.1666666731|
+-----------------+



In [111]:
# Group by tailnum
by_plane = sdf_flights.groupBy("tailnum")

# Number of flights each plane made
by_plane.count().show(10)

+-------+-----+
|tailnum|count|
+-------+-----+
| N513UA|  102|
| N510UW|   48|
| N8390A|   31|
| N3CWAA|   68|
| N73283|  110|
| N369NB|  187|
| N396AA|   21|
| N8322X|   15|
| N3AEMQ|  276|
| N4YUAA|   42|
+-------+-----+
only showing top 10 rows



In [112]:
# Group by origin
by_origin = sdf_flights.groupBy("origin")

# Average duration of flights from PDX and SEA
by_origin.avg("air_time").show()

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   LGA|117.82580581372355|
|   EWR|153.30002475944914|
|   JFK| 178.3490497712667|
+------+------------------+



## Grouping and Aggregating

In [114]:
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F

# Group by month and dest
by_month_dest = sdf_flights.groupBy('month', 'dest')

# Average departure delay by month and destination
by_month_dest.agg(F.avg('dep_delay')).show()

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|    1| EYW|               13.0|
|   10| CLE|  4.405172413793103|
|   10| JAX|  9.313807531380753|
|   10| BHM| 24.153846153846153|
|   10| DAY| 15.118110236220472|
|   11| OKC|   8.10344827586207|
|   10| DCA|  4.209424083769633|
|   10| DFW|  3.522948539638387|
|   10| OMA|  13.39080459770115|
|   10| MHT|  13.80722891566265|
|   11| LAS|  4.782700421940929|
|    1| MSP|  11.76172607879925|
|   10| BTV| 2.5508474576271185|
|   10| MEM|  7.928104575163399|
|   11| BHM|  19.61904761904762|
|   12| MEM| 31.747899159663866|
|   12| ILM|              31.25|
|   11| HNL|0.18181818181818182|
|    1| SLC|  8.360406091370558|
|   10| TUL| 30.153846153846153|
+-----+----+-------------------+
only showing top 20 rows



In [115]:
# Standard deviation of departure delay
by_month_dest.agg(F.stddev('dep_delay')).show()

+-----+----+----------------------+
|month|dest|stddev_samp(dep_delay)|
+-----+----+----------------------+
|    1| EYW|                  null|
|   10| CLE|    25.820811853116663|
|   10| JAX|    33.089734402632345|
|   10| BHM|     46.98058518809003|
|   10| DAY|     38.15998364721879|
|   11| OKC|     19.23416756635034|
|   10| DCA|    29.070946167440905|
|   10| DFW|     25.93755548637475|
|   10| OMA|     37.85663172581741|
|   10| MHT|    31.988358336380983|
|   11| LAS|    26.271577988930474|
|    1| MSP|     42.05931944173929|
|   10| BTV|     20.65917856191837|
|   10| MEM|      26.8930886678019|
|   11| BHM|    26.433834739734962|
|   12| MEM|      52.2012433268701|
|   12| ILM|      54.0333401878123|
|   11| HNL|    15.146312237992932|
|    1| SLC|     33.21385646170493|
|   10| TUL|     43.20341866814922|
+-----+----+----------------------+
only showing top 20 rows



## Joining